In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, LxF, KP07, CDKLM16, DataOutput, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [2]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [3]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

In [4]:
#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
#Centered in time, centered in space
nx = 100
ny = 200

dx = 200.0
dy = 200.0

width = nx * dx
height = ny * dy

dt = 1
g = 9.81
r = 0.0
A = 1
f = 0.01

#wind = Common.WindStressParams(type=0, tau0=3.0, rho=1025, alpha=1.0/(100*dx))
wind = Common.WindStressParams(type=99)

    

bcSettings = 1
ghosts = [1,1,1,1] # north, east, south, west
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # periodic north-south
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # periodic east-west
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 60;
#addTopographyBump(h0, nx, ny, dx, dy, ghosts, 40)

eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+1+2), dtype=np.float32, order='C');
v0 = np.zeros((ny+1+2, nx+2), dtype=np.float32, order='C');

#Create bump in to lower left of domain for testing
x_center = dx*nx*0.3
y_center = dy*ny*0.2
makeCentralBump(eta0, 0.0, nx, ny, dx, dy, ghosts)
#makeUpperCornerBump(eta0, nx, ny, dx, dy, ghosts)
#addDualVortexStaggered(eta0, u0, v0, nx, ny, dx, dy, ghosts)
    

#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions )


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

ghosts = [-1,1,-1,1]

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[1:-1, 1:-1], u0[1:-1, :], v0[:, 1:-1])

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

#anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
#plt.close(anim._fig)
#anim

reload(DataOutput)
with DataOutput.CTCSNetCDFWriter("hotstart_test.nc", nx, ny, dx, dy, H=h0, num_layers=1, ignore_ghostcells=True, \
                                 width=width, height=height) as writer:
    eta1, u1, v1 = sim.download()
    
    for i in range(30):
        if (i>0):
            t = sim.step(360.0)
        else:
            t = 0.0
        eta1, u1, v1 = sim.download()

        print "i: " + str(i) + " t: " + str(t)
        writer.write(i, t, eta1, u1, v1)